## Importing Dependencies and Merging the Main Dataframe

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data = "Resources/schools_complete.csv"
student_data = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_df = pd.read_csv(school_data)
student_df = pd.read_csv(student_data)

# Combine the data into a single dataset.  
complete_school_df = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])

#view dataset
#complete_school_df.head()

## District Performance Analysis

In [2]:
#checking for any missing data
#complete_school_df.count()

In [3]:
#creating bins for reading scores
reading_bins = [0, 69, 100]

# Create the names for the pass/fail reading scores
reading_pass = ["Fail", "Pass"]

#add to dataframe
complete_school_df["Reading Pass/Fail"] = pd.cut(complete_school_df["reading_score"], 
                                              bins= reading_bins, labels=reading_pass, include_lowest=True)
math_bins = [0, 69, 100]

# Create the names for the pass/fail math scores
math_pass = ["Fail", "Pass"]

#add to dataframe
complete_school_df["Math Pass/Fail"] = pd.cut(complete_school_df["math_score"], 
                                              bins= math_bins, labels=math_pass, include_lowest=True)

#view modified dataframe
#complete_school_df

In [4]:
#counting the number of schools
school_count = len(complete_school_df["school_name"].unique())

#view computed variable
#school_count


In [5]:
#counting the number of students
number_students = complete_school_df["Student ID"].count()
#print(number_students)

#summing the budget
budget = sum(complete_school_df["budget"].unique())
#print(budget)

#computing average math scores
avg_math = round(complete_school_df["math_score"].mean(),2)
#print(avg_math)

#computing average reading scores
avg_reading = round(complete_school_df["reading_score"].mean(),2)
#print(avg_reading)

In [6]:
#determining the number of students who passed math
pass_math = sum(complete_school_df["math_score"] >= 70)

#determining the number of students who passed reading
pass_reading = sum(complete_school_df["reading_score"] >= 70)

In [7]:
#percentages for passing math, reading, and overall (both math and reading)
perc_pass_math = round((pass_math/number_students)*100,2)

perc_pass_reading = round((pass_reading/number_students)*100,2)

overall_percent = round(((perc_pass_math + perc_pass_reading)/2),2)

#view variable
#overall_percent


In [8]:
#obtaining the column names for the dataframe
#complete_school_df.columns

In [9]:
#Creating and Printing the District Summary Table
district_summary_df = pd.DataFrame({"Total Schools": [school_count],
                              "Total Students": [number_students],
                                    "Total Budget" : budget,
                              "Average Math Score": avg_math,
                              "Average Reading Score": avg_reading,
                                   "% Passing Math" : perc_pass_math,
                                   "% Passing Reading" : perc_pass_reading,
                                   "% Overall Passing" : overall_percent})

#view summary table before formatting is applied
#district_summary_df

## District Summary Table

In [10]:
#applying formatting using the mapping function

district_summary_df["Total Budget"] = district_summary_df["Total Budget"].astype(float).map("${:,.2f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].astype(float).map("{:,.2f}%".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].astype(float).map("{:,.2f}%".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].astype(float).map("{:,.2f}%".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",78.99,81.88,74.98%,85.81%,80.40%


## School Performance Analysis

In [11]:
grouped_school_df = complete_school_df.loc[:, ["school_name", "type", "Student ID", "budget", "math_score", "reading_score", 
                                    "Math Pass/Fail", "Reading Pass/Fail"]]
#view new dataframe
#grouped_school_df.head()

In [12]:
#criteria for passing
passing_math= grouped_school_df["math_score"] >= 70
passing_reading = grouped_school_df["reading_score"] >= 70
overall_pass = (passing_math & passing_reading)

#adding passing rates to grouped school dataframe
grouped_school_df["% Passing Math"] = round(passing_math*100,2)
grouped_school_df["% Passing Reading"] = round(passing_reading*100,2)
grouped_school_df["% Overall Passing"] = round(overall_pass*100,2)

#view updated dataframe
#grouped_school_df

In [13]:
#determining school budget
school_budget = grouped_school_df.groupby(["school_name"])["budget"].mean()
#print(school_budget)

#determining number of students
school_students = grouped_school_df.groupby(["school_name"])["Student ID"].count()
#print(school_students)

In [14]:
#budget per student
budget_per_student = school_budget / school_students

#view variable
#budget_per_student

In [15]:
#creating budget sub dataframe
budget_sub_df = pd.DataFrame({"Per Student Budget" : budget_per_student})

#view dataframe
#budget_sub_df

In [16]:
#merge budget sub datadrame with the grouped school dataframe
grouped_school_df2 = pd.merge(grouped_school_df, budget_sub_df, how="left", on=["school_name", "school_name"])

#view dataframe
#grouped_school_df2.head()

In [17]:
#renaming columns
grouped_school_df3= grouped_school_df2.rename(columns=
                                              {"school_name" : "School Name",
                                               "type" : "School Type",
                                               "Student ID" : "Total Students", 
                                               "budget" : "School Budget",
                                               "Per Student Budget" : "Per Student Budget",
                                               "math_score" : "Average Math Score", 
                                               "reading_score": "Average Reading Score", 
                                              "% Passing Math" : "% Passing Math",
                                              "% Passing Reading" : "% Passing Reading",
                                              "% Overall Passing" : "% Overall Passing"})

#view datafame
#grouped_school_df3.head()

In [18]:
#grouping by schools to create a dataframe with average values
school_summary_df = grouped_school_df3.groupby(["School Name"])

school_summary_df2= school_summary_df[["School Budget", "Per Student Budget", 
                                      "Average Math Score", "Average Reading Score",
                                       "% Passing Math", "% Passing Reading",
                                       "% Overall Passing"]].mean()

#view dataframe
#school_summary_df2

In [19]:
#grouping by schools to create a dataframe for the total number of students
school_summary_df = grouped_school_df3.groupby(["School Name"])

school_summary_df3 = pd.DataFrame(school_summary_df['Total Students'].count())

#view dataframe
#school_summary_df3

In [20]:
#merging the two dataframes to create an aggregated dataframe
agg_school_summary = pd.merge(school_summary_df2, school_summary_df3, how="left", left_index=True, right_index=True)

#view dataframe
#agg_school_summary.head(15)

In [21]:
#obtaining information about school type
school_df2 = school_df.groupby(["school_name"])

#view the school names
#school_df2["type"].unique()

In [22]:
#creating a dataframe with only school name and school type
name_type_df = pd.DataFrame({
    "School Name": ["Bailey High School", "Cabrera High School", "Figueroa High School", "Ford High School", "Griffin High School",
                    "Hernandez High School", "Holden High School", "Huang High School", "Johnson High School", "Pena High School",
                   "Rodriguez High School", "Shelton High School", "Thomas High School", "Wilson High School", "Wright High School"],
    "School Type": ["District", "Charter", "District", "District", "Charter", 
                    "District", "Charter", "District", "District", "Charter",
                   "District", "Charter", "Charter", "Charter", "Charter"]
})

#view dataframe
#name_type_df

In [23]:
#merging with aggregated dataframe
agg_school_summary2 = pd.merge(agg_school_summary, name_type_df, how="left", on=["School Name", "School Name"])

#view updated dataframe
#agg_school_summary2.head(15)

In [24]:
#saving this table in a separate dataframe
#this way, we can apply formatting to this new dataframe while leaving the other one alone for computational purposes
agg_school_summary_3 = agg_school_summary2

#view dataframe before formatting is applied
#agg_school_summary_3

## School Performance Summary

In [25]:
#applying formatting using the mapping function

agg_school_summary_3["School Budget"] = agg_school_summary_3["School Budget"].astype(float).map("${:,.2f}".format)
agg_school_summary_3["Per Student Budget"] = agg_school_summary_3["Per Student Budget"].astype(float).map("${:,.2f}".format)
agg_school_summary_3["Average Math Score"] = agg_school_summary_3["Average Math Score"].astype(float).map("{:,.2f}".format)
agg_school_summary_3["Average Reading Score"] = agg_school_summary_3["Average Reading Score"].astype(float).map("{:,.2f}".format)
agg_school_summary_3["% Passing Math"] = agg_school_summary_3["% Passing Math"].astype(float).map("{:,.2f}%".format)
agg_school_summary_3["% Passing Reading"] = agg_school_summary_3["% Passing Reading"].astype(float).map("{:,.2f}%".format)
agg_school_summary_3["% Overall Passing"] = agg_school_summary_3["% Overall Passing"].astype(float).map("{:,.2f}%".format)

agg_school_summary_3

,School Name,School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Total Students,School Type
0,Bailey High School,"$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%,4976,District
1,Cabrera High School,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%,1858,Charter
2,Figueroa High School,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%,2949,District
3,Ford High School,"$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%,2739,District
4,Griffin High School,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%,1468,Charter
5,Hernandez High School,"$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%,4635,District
6,Holden High School,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,89.23%,427,Charter
7,Huang High School,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%,2917,District
8,Johnson High School,"$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%,4761,District
9,Pena High School,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%,962,Charter


## Top Performing Schools Analysis & Summary

In [26]:
# sorting % Overall Passing in descending order to determine the top five performing schools
performance = agg_school_summary2.sort_values(["% Overall Passing"], ascending = False)

top_five_df = performance.head(5)

#view dataframe
top_five_df

,School Name,School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Total Students,School Type
1,Cabrera High School,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%,1858,Charter
12,Thomas High School,"$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,90.95%,1635,Charter
4,Griffin High School,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%,1468,Charter
13,Wilson High School,"$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%,2283,Charter
9,Pena High School,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%,962,Charter


## Bottom Performing Schools Analysis & Summary

In [27]:
# sorting % Overall Passing in descending order to determine the top five performing schools
performance2 = agg_school_summary2.sort_values(["% Overall Passing"], ascending = True)

bottom_five_df = performance2.head(5)

#view dataframe
bottom_five_df


,School Name,School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Total Students,School Type
10,Rodriguez High School,"$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,52.99%,3999,District
2,Figueroa High School,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%,2949,District
7,Huang High School,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%,2917,District
5,Hernandez High School,"$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%,4635,District
8,Johnson High School,"$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%,4761,District


## Math Scores By Grade Analysis

In [28]:
#creating math dataframe
grade_math_df = complete_school_df.loc[:, ["grade", "math_score"]]

#view dataframe
grade_math_df.head()

,grade,math_score
0,9th,79
1,12th,61
2,12th,60
3,12th,58
4,9th,84


In [29]:
#group math scores by grade
grade_math_df2 = grade_math_df.groupby(["grade"])

grade_math_df3 = round(grade_math_df2.mean(),2)

#view dataframe
grade_math_df3

,math_score
grade,
10th,78.94
11th,79.08
12th,78.99
9th,78.94


## Math Scores By Grade Summary

In [30]:
math_summary_df= grade_math_df3.rename(columns=
                                              {"grade" : "Grade",
                                               "math_score" : "Average Math Score"
                                               })

math_summary_df

,Average Math Score
grade,
10th,78.94
11th,79.08
12th,78.99
9th,78.94


## Reading Scores By Grade Analysis

In [31]:
#creating reading dataframe
grade_reading_df = complete_school_df.loc[:, ["grade", "reading_score"]]

#view dataframe
#grade_reading_df.head()

In [32]:
#group reading scores by grade
grouped_grade_read_df = grade_reading_df.groupby(["grade"])

reading_summary_df = round(grouped_grade_read_df.mean(),2)

#view dataframe
#reading_summary_df

## Reading Scores By Grade Summary

In [33]:
reading_summary_2= reading_summary_df.rename(columns=
                                              {"grade" : "Grade",
                                               "reading_score" : "Average Reading Score"
                                               })

reading_summary_2

,Average Reading Score
grade,
10th,81.87
11th,81.89
12th,81.82
9th,81.91


## Scores By School Spending Analysis

In [34]:
#creating data frame for school spending
school_spending_df = grouped_school_df3.loc[:, ["Average Math Score", "Average Reading Score", 
                                                "% Passing Math", "% Passing Reading",
                                                "% Overall Passing", "Per Student Budget"]]
#view dataframe
#school_spending_df.head()


In [35]:
#bin for school spending
spending_bins = [0, 585, 630, 645, 680]

#names for the bins
budget_labels = ["<584", "585-630", "631-645", "646-680"]

school_spending_df["Per Student Budget"] = pd.cut(school_spending_df["Per Student Budget"],
                                              bins= spending_bins, labels=budget_labels, include_lowest=True)

#view dataframe
#school_spending_df


In [36]:
school_spending_df2 = school_spending_df.groupby(["Per Student Budget"])

school_spending_summary_df = round(school_spending_df2.mean(),2)

#view dataframe
#school_spending_summary_df

## Scores By School Spending Summary

In [37]:
#applying appropriate formatting to summary table
school_spending_summary_df["% Passing Math"] = school_spending_summary_df["% Passing Math"].astype(float).map("{:,.2f}%".format)
school_spending_summary_df["% Passing Reading"] = school_spending_summary_df["% Passing Reading"].astype(float).map("{:,.2f}%".format)
school_spending_summary_df["% Overall Passing"] = school_spending_summary_df["% Overall Passing"].astype(float).map("{:,.2f}%".format)

school_spending_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Per Student Budget,,,,,
<584,83.36,83.96,93.70%,96.69%,90.64%
585-630,79.98,82.31,79.11%,88.51%,70.94%
631-645,77.82,81.30,70.62%,82.60%,58.84%
646-680,77.05,81.01,66.23%,81.11%,53.53%


## Scores By School Size Analysis

In [38]:
school_size_df = pd.merge(grouped_school_df2, complete_school_df, how="left", on=["Student ID", "Student ID"])

#view dataframe
#school_size_df.head()

In [39]:
school_size_df2 = school_size_df.loc[:, ["math_score_x", "reading_score_x", 
                                                "% Passing Math", "% Passing Reading",
                                                "% Overall Passing", "size"]]
#view dataframe
#school_size_df2.head()

In [40]:
#bin in sizes school sizes small, medium, and large
size_bins = [0, 1000, 2000, 3000]

#labels for the size bins
size_labels = ["Small (<1000)", "Medium (1000-1999)", "Large (2000-5000)"]

school_size_df2["size"] = pd.cut(school_size_df2["size"], 
                                              bins= size_bins, labels=size_labels, include_lowest=True)

#view dataframe
#school_size_df2

In [41]:
#renaming columns
school_size_df3= school_size_df2.rename(columns=
                                              {"math_score_x" : "Average Math Score",
                                               "reading_score_x" : "Average Reading Score",
                                               "% Passing Math" : "% Passing Math",
                                               "% Passing Reading" : "% Passing Reading",
                                               "% Overall Passing" : "% Overall Passing",
                                               "size" :"School Size"
                                               })

#view dataframe
#school_size_df3

In [42]:
school_size_df4 = school_size_df3.groupby(["School Size"])

school_size_summary_df = round(school_size_df4.mean(),2)

#view dataframe
#school_size_summary_df


## Scores By School Size Summary

In [43]:
#applying appropriate formatting to summary table
school_size_summary_df["% Passing Math"] = school_size_summary_df["% Passing Math"].astype(float).map("{:,.2f}%".format)
school_size_summary_df["% Passing Reading"] = school_size_summary_df["% Passing Reading"].astype(float).map("{:,.2f}%".format)
school_size_summary_df["% Overall Passing"] = school_size_summary_df["% Overall Passing"].astype(float).map("{:,.2f}%".format)

school_size_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.83,83.97,93.95%,96.04%,90.14%
Medium (1000-1999),83.37,83.87,93.62%,96.77%,90.62%
Large (2000-5000),78.16,81.65,72.34%,83.84%,61.40%


### Scores by School Type Analysis

In [44]:
school_type_df = grouped_school_df3.loc[:, ["Average Math Score", "Average Reading Score", 
                                                "% Passing Math", "% Passing Reading",
                                                "% Overall Passing", "School Type"]]
#view dataframe
#school_type_df.head()

In [45]:
school_type_df2 = school_type_df.groupby(["School Type"])

school_type_summary_df = round(school_type_df2.mean(),2)

#view dataframe
#school_type_summary_df

### Scores by School Type Summary

In [46]:
#applying appropriate formatting to summary table
school_type_summary_df["% Passing Math"] = school_type_summary_df["% Passing Math"].astype(float).map("{:,.2f}%".format)
school_type_summary_df["% Passing Reading"] = school_type_summary_df["% Passing Reading"].astype(float).map("{:,.2f}%".format)
school_type_summary_df["% Overall Passing"] = school_type_summary_df["% Overall Passing"].astype(float).map("{:,.2f}%".format)

school_type_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.41,83.90,93.70%,96.65%,90.56%
District,76.99,80.96,66.52%,80.91%,53.70%
